In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.model_impl.combined_model import WCMSLModel



In [4]:
model = WCMSLModel()
model.load(PROJECT_ROOT / "src" / "models" / "combined_model.joblib")

In [10]:
while (a := input("Enter text (or 'exit' to quit): ")) not in ("exit", ''):
    prediction = model.predict(a)
    print(f"Prediction: {prediction}")

Prediction: Result: AI,
Probability HUMAN: 0.1325,
Probability AI: 0.8675
Prediction: Result: AI,
Probability HUMAN: 0.0001,
Probability AI: 0.9999
Prediction: Result: HUMAN,
Probability HUMAN: 0.8160,
Probability AI: 0.1840
Prediction: Result: AI,
Probability HUMAN: 0.0000,
Probability AI: 1.0000
Prediction: Result: AI,
Probability HUMAN: 0.0000,
Probability AI: 1.0000
Prediction: Result: HUMAN,
Probability HUMAN: 0.2604,
Probability AI: 0.7396
Prediction: Result: AI,
Probability HUMAN: 0.0059,
Probability AI: 0.9941
Prediction: Result: HUMAN,
Probability HUMAN: 0.9845,
Probability AI: 0.0155
Prediction: Result: HUMAN,
Probability HUMAN: 0.9611,
Probability AI: 0.0389
Prediction: Result: AI,
Probability HUMAN: 0.0392,
Probability AI: 0.9608
Prediction: Result: AI,
Probability HUMAN: 0.0087,
Probability AI: 0.9913
Prediction: Result: HUMAN,
Probability HUMAN: 0.3578,
Probability AI: 0.6422


In [10]:
import pandas as pd

from tqdm import tqdm

df1 = pd.read_csv(PROJECT_ROOT / "data" / "processed" / "balanced_length_filtered_test_dataset.csv")

all = len(df1)
correct = 0

for _, row in tqdm(df1.iterrows(), total=all):
    text, ai = row['text'], row['generated']
    prediction = model.predict(text).probability_ai

    if prediction > 0.5:
        prediction = 1
    else:
        prediction = 0

    if prediction == ai:
        correct += 1

accuracy = correct / all
print(f"Accuracy on AI Generated Essays Dataset: {accuracy:.4f}")

100%|██████████| 3000/3000 [00:18<00:00, 162.02it/s]

Accuracy on AI Generated Essays Dataset: 0.9997
